In [23]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests # library to handle requests
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium

#! conda install -c conda-forge ipywidgets --yes
import ipywidgets as widgets

print('Folium installed and imported!')

Folium installed and imported!


In [24]:
# @hidden_cell
CLIENT_ID = ' ' # your Foursquare ID
CLIENT_SECRET = ' ' # your Foursquare Secret
VERSION = '20200501'
LIMIT = 100 # max only 50 result 

radius = 10000

In [25]:
delhi_district_file_path='./delhi_all.csv'
delhi_district=pd.read_csv(delhi_district_file_path)

delhi_population_path='./delhi9_population.csv'
delhi_population=pd.read_csv(delhi_population_path)

delhi_geo_path='./Delhi.geojson'
import json
#delhi_geo = json.loads(delhi_geo_path)
with open(delhi_geo_path, 'r') as j:
     delhi_geo = json.loads(j.read())
        
#delhi_geo=pd.read_csv(delhi_geo_path, error_bad_lines=False)

In [26]:
 delhi_district.place_name.unique()
# delhi_population.head(3)
# delhi_geo.columns
# print(delhi_geo)

array(['Connaught Place', 'Darya Ganj', 'Aliganj', 'Rashtrapati Bhawan',
       'Lower Camp Anand Parbat', 'Bara Tooti', 'Birla Lines',
       'Patel Nagar', 'Delhi Cantt', 'Nirman Bhawan', 'Inderpuri',
       'Hazrat Nizamuddin', 'Jangpura', 'Zakhira', 'Hauz Khas',
       'Malviya Nagar', 'Vishnu Garden', 'Nehru Place',
       'Flatted Factories Complex', 'Malcha Marg', 'Postal Saving Bureau',
       'Kidwai Nagar', 'Lajpat Nagar', 'Jamia Nagar', 'Punjabi Bagh',
       'J 6block Rajouri Garden', 'Naraina Industrial Estate',
       'Himayunpur Extn', 'T B Hospital', 'Gandhi Nagar', 'Shahdara',
       'Adarsh Nagar', 'Pitampura', 'Inderlok', 'Alipur', 'Gurgaon Road',
       'A F Rajokari', 'Bawana', 'Sanoth', 'Nagloi', 'Badli', 'Najafgarh',
       'Badarpur T P Station', 'Palam Enclave', 'Nangal Rava',
       'Arjan Garh', 'Kailash', 'Andrews Ganj', 'Azad Nagar',
       'Wazirpur Phase Iii', 'Zafrabad', 'Civil Lines', 'Paharganj',
       'Shakurbasti', 'Munirka', 'Janakpuri', 'Uttam Nag

In [27]:
from IPython.display import clear_output
caption=widgets.Label(value='Not selected yet')
ALL='ALL'
def unique_sorted_values_ALL(array):
    unique=array.unique().tolist()
    unique.sort()
    unique.insert(0,ALL)
    return unique
x='JAMES'
def onchange(d):
    #clear_output(wait=False)
    #print ('%s' % d['new'] )
    pass
    #print('Selection changed event fired.')

district=widgets.Dropdown(
    options=delhi_district.place_name,
    description='Places :'
)
widgets.link((district,'value'),(caption,'value')) 
district.observe(onchange)
display(district, caption) 

Dropdown(description='Places :', options=('Connaught Place', 'Darya Ganj', 'Aliganj', 'Rashtrapati Bhawan', 'L…

Label(value='Connaught Place')

In [28]:
vars(district); 
state=district.value
selectedRow=delhi_district.loc[delhi_district['place_name']==district.value, :]

vars(selectedRow);

In [29]:
geolocator = Nominatim(user_agent="foursquare_agent", format_string="%s, New delhi,IN")
# location = geolocator.geocode(district.value)
# location = geolocator.reverse("28.509669, 77.376294")
# latitude = location.latitude
# longitude = location.longitude

latitude = selectedRow.loc[district.index,'latitude']
longitude =selectedRow.loc[district.index,'longitude']
location = geolocator.reverse("{}, {}".format(latitude, longitude))
 
# location.raw

In [30]:
# Delhi latitude and longitude values
delhi_map=folium.Map(location=[latitude, longitude], zoom_start=10.47)

# Add the color for the chloropleth:
delhi_map.choropleth(
 geo_data=delhi_geo,
 name='choropleth',
 data=delhi_population,
 columns=['District', 'Percentage'],
 key_on='feature.properties.Dist_Name',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Population (%)'
)
# folium.LayerControl().add_to(delhi_map)


# I can add marker one by one on the map
for i in range(0,len(delhi_district)):
    folium.Marker([delhi_district.iloc[i]['latitude'], delhi_district.iloc[i]['longitude']], popup=folium.Popup(delhi_district.iloc[i]['place_name'], parse_html=True, max_width=100) ).add_to(delhi_map)

delhi_map

In [31]:
search_query = 'market'
url = 'https://api.foursquare.com/v2/search/recommendations?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}&state={}&radius={}&sortByDistance=True'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, LIMIT, state, radius)
url
# search/recommendations

'https://api.foursquare.com/v2/search/recommendations?client_id= &client_secret= &ll=28.6333,77.2167&v=20200501&query=market&limit=100&state=Connaught Place&radius=10000&sortByDistance=True'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5ee5013a08823562b21e38d3'},
 'response': {}}

In [33]:
# assign relevant part of JSON to venues
venues = results['response']['group']['results']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe=dataframe[dataframe['venue.location.distance']<=5000]

dataframe.head()
# dataframe['location.address']


KeyError: 'group'